<a href="https://colab.research.google.com/github/rushivarun/Short_circuit_analysis/blob/master/Shortcircuit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Power System Engineering**
## Digital Assignment 2

### Submitted by
#### **Muskaan Singh 17BEE0312**
#### **Shivram Sugavanam 17BEE0132**
#### **Rushi Varun 17BEE0285**






**TASK**
### Using any software simulation tool, Investigate the nature of fault and location of fault for any sample power system of your choice. Give your inference on the above investigation.

The following is a IPython notebook aiming to solve the above given task.

In [0]:
# install necessary dependencies
# panda power is a python module for simuating power system locally
!pip install pandapower

     |████████████████████████████████| 4.5MB 3.5MB/s 
  Created wheel for pandapower: filename=pandapower-2.1.0-cp36-none-any.whl size=4434079 sha256=62431b4358d076aeeb631614b61864f08343e5607ed62209d5108489814b8484
  Stored in directory: /root/.cache/pip/wheels/f8/34/f9/9a3f4b424443ffed9c0840ab4903c6d6181da90c544ccd5210
Successfully built pandapower


First, we create a simple open ring network with 4 buses, that are connected by one transformer and two lines with one open sectioning point. The network is fed by an external grid connection at bus 1:



In [0]:
import pandapower as pp
import pandapower.shortcircuit as sc

def ring_network():
    net = pp.create_empty_network()
    b1 = pp.create_bus(net, 220)
    b2 = pp.create_bus(net, 110)
    b3 = pp.create_bus(net, 110)
    b4 = pp.create_bus(net, 110)
    pp.create_ext_grid(net, b1, s_sc_max_mva=100., s_sc_min_mva=80., rx_min=0.20, rx_max=0.35)
    pp.create_transformer(net, b1, b2, "100 MVA 220/110 kV")
    pp.create_line(net, b2, b3, std_type="N2XS(FL)2Y 1x120 RM/35 64/110 kV" , length_km=15.)
    l2 = pp.create_line(net, b3, b4, std_type="N2XS(FL)2Y 1x120 RM/35 64/110 kV" , length_km=12.)
    pp.create_line(net, b4, b2, std_type="N2XS(FL)2Y 1x120 RM/35 64/110 kV" , length_km=10.)
    pp.create_switch(net, b4, l2, closed=False, et="l")
    return net

# Symmetric Short-Circuit Calculation
Maximum Short Circuit Currents
Now, we load the network and calculate the maximum short-circuit currents with the calc_sc function:

In [0]:
net = ring_network()
sc.calc_sc(net, case="max", ip=True, ith=True, branch_results=True)
net.res_bus_sc


Branch results are in beta mode and might not always be reliable, especially for transformers


,ikss_ka,ip_ka,ith_ka
0,0.262432,0.505834,0.263723
1,0.476454,0.942589,0.479039
2,0.466671,0.915418,0.469123
3,0.469892,0.924301,0.472386


where ikss is the initial short-circuit current, ip is the peak short-circuit current and ith is the thermal equivalent current.

For branches, the results are defined as the maximum current flows through that occurs for a fault at any bus in the network. The results are available seperately for lines:

In [0]:
net.res_line_sc

,ikss_ka,ip_ka,ith_ka
0,0.466671,0.915418,0.469123
1,0.459100,0.894771,0.461455
2,0.469892,0.924301,0.472386


transformers are....

In [0]:
net.res_trafo_sc


,ikss_hv_ka,ikss_lv_ka
0,0.238227,0.476454


# Minimum Short Circuit Currents
Minimum short-circuits can be calculated in the same way. However, we need to specify the end temperature of the lines after a fault as per standard first:

In [0]:
net = ring_network()
net.line["endtemp_degree"] = 80
sc.calc_sc(net, case="min", ith=True, ip=True, branch_results=True)
net.res_bus_sc

Branch results are in beta mode and might not always be reliable, especially for transformers


,ikss_ka,ip_ka,ith_ka
0,0.209946,0.462534,0.211736
1,0.384422,0.860874,0.387974
2,0.377608,0.832832,0.380846
3,0.379861,0.841982,0.383197


The branch results are now the minimum current flows through each branch:



In [0]:
net.res_line_sc


,ikss_ka,ip_ka,ith_ka
0,0.372278,0.811639,0.375302
1,0.372278,0.811639,0.375302
2,0.379861,0.841982,0.383197


In [0]:
net.res_trafo_sc


,ikss_hv_ka,ikss_lv_ka
0,0.186139,0.372278


# Fault Impedance
It is also possible to specify a fault impedance in the short-circuit calculation:

In [0]:
net = ring_network()
sc.calc_sc(net, topology="radial", ip=True, ith=True, r_fault_ohm=1., x_fault_ohm=2.)

In [0]:
net.res_bus_sc


,ikss_ka,ip_ka,ith_ka
0,0.261343,0.503509,0.262627
1,0.469382,0.926656,0.471909
2,0.459875,0.900379,0.462274
3,0.463005,0.908972,0.465446
